In [ ]:
# setup-- 
import os
import pyspark
from splicemachine.spark.context import PySpliceContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

# make sure pyspark tells workers to use python3 not 2 if both are installed
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
jdbc_host = os.environ['JDBC_HOST']

conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

splicejdbc=f"jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=admin"

splice = PySpliceContext(spark, splicejdbc)



# Splice Transactions with Spark and JDBC

Now we'll get a glimpse into the transactional nature of Splice Machine.  We'll be switching to the `%spark` interpreter, which uses the Scala programming language; this allows us to easily demonstrate a clean way to programmatically interact with Splice Machine via JDBC. In this demonstration, we'll:

<ul class="italic">
    <li>Run setup code to create our variables and define a simple Spark procedure named <code>checkBalance</code>.</li>
    <li>Use JDBC calls to set and check values from the database.</li>
    <li>Show the use of <code>commit</code> and <code>rollback</code> to control transactional persistence of our updates.</li>
</ul>
<br />
## Run Setup Code and Define our Procedure

We define a simple procedure, `checkBalance` that checks and *pretty-prints* the current balance for a supplier  

### In the cell below, input your JDBCurl as well as your user and password in the <code>defaultJDBCURL</code> variable

In [ ]:
%%scala 
println("Please copy and paste your JDBC URL. You can find it at the bottom right of your cluster dashboard")
val defaultJDBCURL = z.input("JDBCurl","""jdbc:splice://{FRAMEWORKNAME}-proxy.marathon.mesos:1527/splicedb;user=splice;password=admin""").toString
val localJDBCURL = """jdbc:splice://localhost:1527/splicedb;user=splice;password=admin"""



In [ ]:
%%scala 
import java.sql.{DriverManager,Connection}
var conn = DriverManager.getConnection(defaultJDBCURL)
conn.setAutoCommit(false)
var statement = conn.createStatement()

def checkBalance(suppKey: Integer) : Unit = {
    var resSet = statement.executeQuery("SELECT S_ACCTBAL FROM TPCH1.SUPPLIER WHERE S_SUPPKEY = " + suppKey.toString())
    while ( resSet.next() ) {
        var bl = resSet.getString("S_ACCTBAL")
        println("================ Current account balance: " + bl + " ====================");
    }
}


Now we can check the balance easily at any time with the checkBalance call, passing in the supply key as a parameter:

In [ ]:
%%scala 
checkBalance(6517)

Our setup code created two useful variables we can now employ:

<table class="splicezepNoBorder">
   <tbody>
      <tr>
          <td class="CodeFont">statement</td>
          <td>The standard JDBC Statement object, on which we run functions such as `executeUpdate`.
      </tr>
      <tr style="background-color: transparent;">
          <td class="CodeFont">conn</td>
          <td style="background-color: transparent;" >A standard JDBC Connection object, on which run `commit` and `rollback`.
      </tr>
    </tbody>
</table>

The first example shows an update statement which, upon committing, retains the value to which it is set.

## Committing a Transaction

In [ ]:
%%scala 
statement.executeUpdate("update TPCH1.SUPPLIER SET S_ACCTBAL = 10000 WHERE S_SUPPKEY = 6517");
checkBalance(6517);
conn.commit();
checkBalance(6517);

<br />
But if we choose to do a rollback, the value returns to the previous value before the transactional context, as shown in the next example.

## Rolling Back a Transaction

In [ ]:
%%scala 
statement.executeUpdate("update TPCH1.SUPPLIER SET S_ACCTBAL = 0 WHERE S_SUPPKEY = 6517")
checkBalance(6517);
conn.rollback();
checkBalance(6517);


## Where to Go Next

The next notebook in this presentation walks you through <a href="./3.5%20Creating%20Applications.ipynb">creating applications with Splice Machine.</a>
